In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
from bs4 import BeautifulSoup

In [5]:
from tqdm import tqdm
def risultati(stagione):
    s_home=[]
    s_away=[]
    g_home=[]
    g_away=[]
    girone=[]
    for g in tqdm(range(1,40)):
        try:
            URL = 'https://www.calcio.com/calendario/ita-serie-a-'+stagione+'-spieltag/'+str(g)+'/'
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, 'html.parser')
        except:
            break
    

        for tr in soup.find_all('tr'):
            tds = tr.find_all('td',{"class": "hell"})
            try:
                home=tds[2].text.replace('\n','')
                away=tds[4].text.replace('\n','')
                ris=tds[5].text.replace('\n','')
                ris=ris.split(':')
                ris_h=ris[0]
                ris_a=ris[1].split('(')[0]
                ris_h=int(ris_h)
                ris_a=int(ris_a)
                #print(home,away,ris_h,ris_a)

                s_home.append(home)
                s_away.append(away)
                g_home.append(ris_h)
                g_away.append(ris_a)
                girone.append(g)
            except:
                pass

        for tr in soup.find_all('tr'):
            tds = tr.find_all('td',{"class": "dunkel"})
            try:
                home=tds[2].text.replace('\n','')
                away=tds[4].text.replace('\n','')
                ris=tds[5].text.replace('\n','')
                ris=ris.split(':')
                ris_h=ris[0]
                ris_a=ris[1].split('(')[0]
                ris_h=int(ris_h)
                ris_a=int(ris_a)
                #print(home,away,ris_h,ris_a)

                s_home.append(home)
                s_away.append(away)
                g_home.append(ris_h)
                g_away.append(ris_a)
                girone.append(g)
            except:
                pass
    
    df=pd.DataFrame()
    df['team_h']=s_home
    df['team_a']=s_away
    df['gol_h']=g_home
    df['gol_a']=g_away
    df['giornata']=girone

    return df

In [19]:
anno=2011
for i in range(8):
    stagione=str(anno+i)+'-'+str(anno+1+i)
    ris=risultati(stagione)
    #ris.to_csv('data/stagione'+stagione+'.csv',index=False)

100%|██████████| 39/39 [00:27<00:00,  1.44it/s]


# Dataset
Ogni partita e' un dato X che contiene 2x3 informazioni a1,a2,a3 per ogni team. La Y e' l'esito, team 1 vince 1, team 2 vince -1

In [6]:
df='data/stagione2013-2014.csv'
df=pd.read_csv(df)
df

,team_h,team_a,gol_h,gol_a,giornata
0,Verona,Milan,2,1,1
1,Inter,Genoa,2,0,1
2,Lazio,Udinese,2,1,1
3,Napoli,Bologna,3,0,1
4,Torino,Sassuolo,2,0,1
...,...,...,...,...,...
375,Catania,Atalanta,2,1,38
376,Genoa,Roma,1,0,38
377,Napoli,Verona,5,1,38
378,Parma,Livorno,2,0,38


In [27]:
dic={}
dic['pippo']=np.array([1,2,3])

In [48]:
def trasforma(s):
    out=[]
    for i in s:
        if i=='v': out.append(3)
        elif i=='p': out.append(1)
        else: out.append(0)
    return out
            
class team:
    def __init__(self,risultati,dr):
        vec=trasforma(risultati)
        print(vec)
        self.a1 = vec[-1]             #ultima partita
        self.a2 = np.sum(vec[-3:])    #ultim tre partite
        self.a3 = np.sum(vec)         #tutte le partite
        self.ultime = risultati       #stringa v=vitt l=sconfitta p=pareggio
        self.dr = dr                  #differenzareti
    def aggiorna(self,risultato,dr):
        self.dr += dr
        self.ultime+=risultato
        self.ultime=self.ultime[1:]
        vec=trasforma(self.ultime)
        self.a1 = vec[-1]          #ultima partita
        self.a2 = np.sum(vec[-3:]) #ultim tre partite
        self.a3 +=self.a1          #tutte le partite
    
    def stampa(self):
        print('a1:',self.a1,'  a2:',self.a2,'  a3:',self.a3,'  diff reti:',self.dr,'  ultimi ris:',self.ultime)
        return


In [49]:
milan=team('lvp',3)

[0, 3, 1]


In [50]:
milan.stampa()

a1: 1   a2: 4   a3: 4   diff reti: 3   ultimi ris: lvp


In [51]:
milan.aggiorna('l',-2)

In [52]:
milan.stampa()

a1: 0   a2: 4   a3: 4   diff reti: 1   ultimi ris: vpl
